In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tsmoothie.smoother import GaussianSmoother
import spikeinterface
import spikeinterface.full as si
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
import spikeinterface.postprocessing as sp
import spikeinterface.preprocessing as spre
import spikeinterface.qualitymetrics as qm
import helper_functions as helper

In [ ]:
import importlib
importlib.reload(sw)


In [2]:
#Reading the file, BP filtering
file_path = '/mnt/disk15tb/mmpatil/Spikesorting/Data/Mandar/230705/18712/Network/000010/data.raw.h5'#network data from chip 16848

recording1 = se.read_maxwell(file_path)



#recording = si.ConcatenateSegmentRecording([recording1,recording2])
channel_ids = recording1.get_channel_ids()
fs = recording1.get_sampling_frequency()
num_chan = recording1.get_num_channels()
num_seg = recording1.get_num_segments()
total_recording = recording1.get_total_duration()

#print('Channel ids:', channel_ids)
print('Sampling frequency:', fs)
print('Number of channels:', num_chan)
print('Number of segments:', num_seg)
print(f"total_recording: {total_recording} s")

recording_bp = spre.bandpass_filter(recording1, freq_min=300, freq_max=6000)

recodring_cmr = spre.common_reference(recording_bp, reference='global', operator='median')
#recording_chunk = recodring_cmr.frame_slice(start_frame= 1*fs,end_frame=425*fs)
recording_chunk = recodring_cmr.frame_slice(start_frame= 0*fs,end_frame=300*fs)

print(f"chunk duration: {recording_chunk.get_total_duration()} s")

Sampling frequency: 20000.0
Number of channels: 1006
Number of segments: 1
total_recording: 300.06 s
chunk duration: 300.0 s


In [4]:
default_KS2_params = ss.get_default_sorter_params('kilosort2')
default_KS2_params['keep_good_only'] = True
default_KS2_params['detect_threshold'] = 12
default_KS2_params['projection_threshold']=[18, 10]
default_KS2_params['preclust_threshold'] = 8
run_sorter = ss.run_kilosort2(recording_chunk, output_folder="/mnt/disk15tb/mmpatil/Spikesorting/sorter_output/sorting_7jul/ks2Samp/", docker_image= "kilosort2-maxwellcomplib:latest",verbose=True, **default_KS2_params)
#run_sorter = ss.run_sorter('kilosort2',recording= recording_chunk, output_folder="/mnt/disk15tb/mmpatil/Spikesorting/sorter_output/kilosort2",docker_image= True,verbose=True, **default_KS2_params)

Starting container
Installing spikeinterface==0.97.1 in kilosort2-maxwellcomplib:latest
Installing extra requirements: ['neo', 'h5py']
Running kilosort2 sorter inside kilosort2-maxwellcomplib:latest
Stopping container


In [4]:
sorting_KS3 = ss.Kilosort2Sorter._get_result_from_folder('/mnt/disk15tb/mmpatil/Spikesorting/sorter_output/sorting_7jul/ks2Samp/sorter_output/')
total_units = sorting_KS3.get_unit_ids()
print(total_units)
print(len(total_units))


[  2   3   4   5   6   7   8   9  11  12  13  14  15  16  18  19  20  21
  25  26  27  28  29  30  31  32  33  36  37  41  42  43  45  46  47  48
  49  50  51  53  54  55  57  59  60  62  63  64  65  66  67  68  69  70
  71  72  73  74  76  77  78  79  80  81  83  84  86  88  93  94  95  96
  97  98 100 101 102 103 106 107 108 109 110 111 112 113 114 116 117 118
 119 120 121 122 123 124 125 127 128 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 146 147 148 149 150 152 153 154 155 156 157 158
 159 161 163 165 166 167 169 172 173 174 175 176 177 178 179 180 181 182
 184 185 188 189 190 191 193 194 195 196 197 199 200 201 202 203 204 206]
162


In [5]:
job_kwargs = dict(n_jobs=64, chunk_duration="1s", progress_bar=True)
#waveforms = si.extract_waveforms(recording_chunk,sorting_KS3,folder="./waveformsblock1_7min",overwrite=True, ms_before=1., ms_after=2.,**job_kwargs)
waveforms = si.extract_waveforms(recording_chunk,sorting_KS3,folder='/mnt/disk15tb/mmpatil/Spikesorting/sorter_output/sorting_7jul/waveforms_Samp',overwrite=True,**job_kwargs)
print(waveforms)

extract waveforms memmap:   0%|          | 0/300 [00:00<?, ?it/s]

WaveformExtractor: 1006 channels - 162 units - 1 segments
  before:60 after:80 n_per_units:500


In [6]:
waveforms = si.load_waveforms('/mnt/disk15tb/mmpatil/Spikesorting/sorter_output/sorting_7jul/waveforms_Samp')#,with_recording=False)
print(waveforms)

WaveformExtractor: 1006 channels - 162 units - 1 segments
  before:60 after:80 n_per_units:500


In [ ]:
waveforms.get_available_extension_names()

In [ ]:
sp.compute_noise_levels(waveforms)

In [7]:
amplitudes = si.compute_spike_amplitudes(waveforms, outputs="by_unit", load_if_exists=True,
                                        **job_kwargs)

extract amplitudes:   0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
waveforms.delete_extension('quality_metrics')

In [ ]:
pc = sp.compute_principal_components(waveforms, n_components = 3,**job_kwargs)

In [ ]:
metrics = waveforms.load_extension("quality_metrics")

In [8]:
import spikeinterface.qualitymetrics as qm
job_kwargs = dict(n_jobs=64, chunk_duration="1s", progress_bar=True)
metrics = qm.compute_quality_metrics(waveforms,**job_kwargs)

/home/mmp/.local/lib/python3.10/site-packages/spikeinterface/qualitymetrics/misc_metrics.py:194: RuntimeWarning: divide by zero encountered in scalar divide
  snrs[unit_id] = np.abs(amplitude) / noise
/home/mmp/.local/lib/python3.10/site-packages/spikeinterface/qualitymetrics/misc_metrics.py:511: UserWarning: Units [18, 21, 32, 46, 54, 55, 57, 86, 88, 110, 111, 119, 128, 134, 139, 143, 153, 159, 163, 165, 169, 175, 181, 185, 191, 200, 201, 206] have too few spikes and amplitude_cutoff is set to NaN
  warnings.warn(f"Units {nan_units} have too few spikes and "


In [9]:
import pandas as pd
pd.set_option('display.max_rows',None)
display(metrics)

,num_spikes,firing_rate,presence_ratio,snr,isi_violations_ratio,isi_violations_count,rp_contamination,rp_violations,sliding_rp_violation,amplitude_cutoff,amplitude_median
2,1827,6.090000,1.0,inf,0.000000,0,0.000000,0,0.035,0.004336,151.062012
3,1604,5.346667,1.0,inf,0.000000,0,0.000000,0,0.040,0.006798,50.354004
4,2530,8.433333,1.0,inf,0.000000,0,0.000000,0,0.010,0.000324,75.531006
5,2955,9.850000,1.0,inf,0.000000,0,0.000000,0,0.005,0.005954,188.827515
6,851,2.836667,1.0,inf,0.000000,0,0.000000,0,0.050,0.017941,62.942505
7,875,2.916667,1.0,inf,0.000000,0,0.000000,0,0.050,0.000887,107.002258
8,802,2.673333,1.0,inf,0.000000,0,0.000000,0,0.055,0.000463,69.236755
9,1333,4.443333,1.0,inf,0.000000,0,0.000000,0,0.020,0.021332,62.942505
11,814,2.713333,1.0,inf,0.000000,0,0.000000,0,0.055,0.036301,81.825256
12,2000,6.666667,1.0,inf,0.000000,0,0.000000,0,0.015,0.000594,62.942505


In [10]:
import mea_analysis_pipeline as msp
unit_ids = msp.remove_violated_units(metrics)
print(unit_ids)

[  2   3   4   5   6   7   8   9  11  12  13  14  15  16  19  20  25  26
  27  28  29  30  31  33  37  41  42  43  45  47  48  49  50  51  53  59
  60  62  63  64  65  66  67  69  70  71  72  73  74  76  77  78  79  80
  81  83  93  94  95  96  97  98 100 101 102 103 106 107 108 109 112 113
 114 116 117 118 120 121 122 123 124 125 127 131 133 135 136 137 138 140
 141 142 144 145 146 147 148 149 150 152 154 155 156 157 158 161 166 167
 172 173 174 176 177 179 180 182 184 188 189 190 193 194 195 196 197 199
 202 203 204]


In [11]:
redundant_units = msp.remove_similar_templates(waveforms)
print(f"redundant-units : {redundant_units}")
non_violated_units = [item for item in unit_ids if item not in redundant_units]

55 30
180 159
206 175
redundant-units : [30, 180, 175]


In [14]:
waveform_good = waveforms.select_units(non_violated_units,new_folder='/mnt/disk15tb/mmpatil/Spikesorting/sorter_output/sorting_7jul/waveforms_goodSA')

In [15]:
waveforms_good = si.load_waveforms('/mnt/disk15tb/mmpatil/Spikesorting/sorter_output/sorting_7jul/waveforms_goodSA')#,with_recording=False)


In [16]:
%matplotlib widget
locations = sp.compute_unit_locations(waveforms_good)
print(len(locations))


127


In [17]:
import pickle

file_path = './SpikeAmplocation.pkl'

with open(file_path,'wb') as file:
    pickle.dump(locations,file)

waveforms.select_units(unit_ids,new_folder='waveforms_good')

In [ ]:
print

In [ ]:
%matplotlib widget
locations = sp.compute_unit_locations(waveforms)
print(type(locations))
import numpy as np
#np.savetxt("unitloc_10mins.txt",locations)
ax = plt.subplot(111)
sw.plot_probe_map(recording1,ax=ax,with_channel_ids=False)
for x,y in locations:
    ax.scatter(x,y)
plt.show()

In [ ]:
import pickle

file_path = './SpikeAmplocation.pkl'

with open(file_path,'wb') as file:
    pickle.dump(locations,file)

In [ ]:
sp.compute_spike_amplitudes(waveforms)

In [ ]:
sp.compute_principal_components(waveforms,n_components=3,mode='by_channel_global')

In [ ]:
from spikeinterface.exporters import export_to_phy
export_to_phy(waveforms, output_folder='/mnt/disk15tb/mmpatil/MEA_Analysis/Python/phy_folder',**job_kwargs)

In [ ]:
!phy template-gui  /mnt/disk15tb/mmpatil/MEA_Analysis/Python/phy_folder/params.py

In [ ]:
metrics = sp.compute_template_metrics(waveforms,load_if_exists=True)
display(metrics)

In [ ]:
matr = sp.compute_template_similarity(waveforms,load_if_exists=True)
# Threshold for similarity
threshold = 0.6

# Find indices where similarity is greater than the threshold
row_indices, col_indices = np.where(matr > threshold)

pairs = [(row_indices[i], col_indices[i]) for i in range(len(row_indices)) if row_indices[i] != col_indices[i]]

# # Print the pairs with similarity > 0.5
# for pair in pairs:
#     print(f"Pair: {pair}")
print(pairs)
plt.imshow(matr, cmap='cool')
plt.colorbar()
plt.show()

In [ ]:
extremum_channels_ids =spikeinterface.full.get_template_extremum_channel(waveforms, peak_sign='neg')
print(extremum_channels_ids)



In [ ]:


print(helper.get_key_by_value(extremum_channels_ids,'625'))

In [ ]:

isi_violations_ratio, isi_violations_count = qm.compute_isi_violations(waveforms, isi_threshold_ms=1.0)
print(isi_violations_ratio)
print(isi_violations_count)

rp_contamination,rp_violation = qm.compute_refrac_period_violations(waveforms)
print(rp_contamination)
print(rp_violation)
914
snr_ratio = qm.compute_snrs(waveforms,peak_sign="both", peak_mode='at_index')
print(snr_ratio)

firing_rate = qm.compute_firing_rates(waveforms)
print(firing_rate)

In [ ]:
import helper_functions as helper


filename = 'Extremechannels_4min.json'
helper.dumpdicttofile(extremum_channels_ids,filename)


In [ ]:
violated_units = [unit for unit, ratio in isi_violations_ratio.items() if ratio > 0.0]
print(violated_units)
print(f"isi violated units:{len(violated_units)}")

refrct_violated_units = [unit for unit,ratio in rp_contamination.items() if ratio >0.0]
print(refrct_violated_units)
print(f"refract vio units:{len(refrct_violated_units)}")

In [ ]:
print(sorting_KS3)

clean_sorting = sorting_KS3.remove_units(refrct_violated_units)
print(clean_sorting)
good_units = [units for units in total_units if units not in refrct_violated_units ]
print(good_units)

#now getting the wavefrom extractor

waveform_good = waveforms.select_units(good_units,new_folder='waveforms_good_100elec')
print(waveform_good)

In [ ]:
%matplotlib widget

In [ ]:
import spikeinterface.postprocessing as sp

locations = sp.compute_unit_locations(waveforms)
print(type(locations))
import numpy as np
#np.savetxt("unitloc_10mins.txt",locations)
ax = plt.subplot(111)
sw.plot_probe_map(recording1,ax=ax,with_channel_ids=False)
for x,y in locations:
    ax.scatter(x,y)

In [ ]:
import spikeinterface_gui
app = spikeinterface_gui.mkQApp() 

# create the mainwindow and show
win = spikeinterface_gui.MainWindow(waveforms)
win.show()
# run the main Qt6 loop
app.exec_()#Need to implement compute noise levels.

In [ ]:
sp.compute_correlograms(waveforms)
sp.compute_spike_amplitudes(waveforms)
w2 = sw.plot_sorting_summary(waveforms, display=False, curation=True, backend="sortingview")

In [ ]:
import kachery_cloud as kcl
kcl.init()

In [ ]:
channel_locations = recording_chunk.get_channel_locations()
channel_ids = recording_chunk.get_channel_ids()
_ = [print(f"{channel_id}: {location}") for location, channel_id in zip(channel_locations, channel_ids)]



In [ ]:
fig, ax1 = plt.subplots(figsize=(15,5))
spike_times = {}
for idx, unit_id in enumerate(clean_sorting.get_unit_ids()):
    spike_train = clean_sorting.get_unit_spike_train(unit_id,start_frame=1*fs,end_frame=100*fs)
    print(spike_train)
    if len(spike_train) > 0:
        spike_times[idx] = spike_train / float(fs)
        #print(spike_times[unit_id])
       # print(unit_id*np.ones_like(spike_times[unit_id]))
        ax1.plot(spike_times[idx],idx*np.ones_like(spike_times[idx]),
                             marker='|', mew=1, markersize=3,
                             ls='',color='black')
                       

In [ ]:
t_start = 0 
t_end = int(600*fs)
dt = 1
#initialising the spike train.
units= clean_sorting.get_num_units()
frame_numbers = t_end
spike_array = np.zeros((units,frame_numbers), dtype= int)
for idx, unit_id in enumerate(clean_sorting.get_unit_ids()):
    spike_train = clean_sorting.get_unit_spike_train(unit_id,start_frame=t_start,end_frame=t_end)
    for spike_time in spike_train:
        spike_array[idx,spike_time] = 1

print(spike_array)

print(spike_array[0,63782])

In [ ]:
np.savez_compressed('spike_array_compressed_blockactivity.npz',spike_array)

In [ ]:
with np.load('spike_array_compressed_blockactivity.npz') as data:
    decompressed_data = data['arr_0']

print(np.array_equal(spike_array, decompressed_data))


In [ ]:
extremum_channels_ids =spikeinterface.full.get_template_extremum_channel(waveforms, peak_sign='neg')
print(extremum_channels_ids)



In [ ]:
colors = [ 'Lime','Gold', 'Orange','Orangered']
fig, ax = plt.subplots()
wf=[]
for i, unit_id in enumerate([34, 49]):
    wf = waveforms.get_waveforms(unit_id)
    color = colors[i]
    ax.plot(wf[:, :,channel_association_dict[594]].T, color=color, lw=0.3)
print(wf.shape)

In [ ]:
colors = [ 'Fuchsia','Olive', 'Teal']
fig, ax = plt.subplots()
wf=[]
for i, unit_id in enumerate([218]):
    wf = waveform_good.get_waveforms(unit_id)
    ax.plot(wf[:, :,channel_association_dict[902]].T, color=colors[0], lw=0.3)
    ax.plot(wf[:, :,channel_association_dict[613]].T, color=colors[1], lw=0.3)
    ax.plot(wf[:, :,channel_association_dict[663]].T, color=colors[2], lw=0.3)
print(wf.shape)

In [ ]:
peak_shift=si.get_template_extremum_channel_peak_shift(waveform_good)

print(peak_shift)

In [ ]:
colors = [ 'Fuchsia','Olive', 'Teal']
fig, ax = plt.subplots()
for i, unit_id in enumerate([26, 40 , 46]):
    template = waveforms.get_template(unit_id)
    color = colors[i]
    ax.plot(template[:, channel_association_dict[780]].T, color=color, lw=3)
print(template.shape)

In [ ]:
w = sw.plot_unit_templates(waveform_good, unit_ids=[183],plot_channels=False )

In [ ]:
w = sw.plot_unit_waveforms(waveform_good, unit_ids=[2,4,7])
w = sw.plot_unit_templates(waveform_good, unit_ids=[2,4,7])
w = sw.plot_unit_probe_map(waveform_good, unit_ids=[2,4,7])